## Autogen Sequential Chats Coding

*[Coding along with the Udemy online course AI Agents: Building Teams of LLM Agents that Work For You by Mohsen Hassan & Ilyass Tabiai]*

Agents can be used to onboard users into a live chat system to solve issues like you can see it on banking webiste or phone provider webistes.

For this example of a sequential chat we will assume that these agents are deployed by a phone provider company called "ACME". The agents are used to gather specifc information about a user that needs help, then to pass that information to a human that will directly have a clear understanding of who requires help, where they're from and what their problem is. The human should be able to immediately act on this information and help the user.

The accomplish this task 3 agents are needed:

An __Onboarding Personal Information Agent__ whose goal it is to get the name and location of the customer.

An __Onboarding Issue Agent__ whose it is goal to determine what the issue of the customer is.

A __Customer Engagement Agent__ who will interact with the customer until a human agent is available for the customer.

An autogen example for Solving Multiple Tasks in a Sequence of Chats can be found at the __[Autogen Docs](https://microsoft.github.io/autogen/docs/notebooks/agentchat_multi_task_chats/)__: The notebook there "showcases how to use the new chat interface of conversational agents in AutoGen: initiate_chats, to conduct a series of tasks. This new interface allows one to pass multiple tasks and their corresponding dedicated agents. Once initiate_chats is invoked, the tasks will be solved sequentially, with the summaries from previous tasks provided to subsequent tasks as context, if the summary_method argument is specified."

In [16]:
from openai import OpenAI
import pandas as pd
from autogen import ConversableAgent

In [17]:
api_key = pd.read_csv("~/tmp/chat_gpt/autogen_agent_1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


In [18]:
# creating a Conversable Agent to accomlpish a simple task
# 1st thing to do is to create a LLM config that specifies which LLM we want to use
# model from the list of models provided by OpenAI https://platform.openai.com/docs/models/continuous-model-upgrades
llm_config = {
    "model": "gpt-4o-mini",
    # "model": "gpt-3.5-turbo",
    "api_key": api_key
    }
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


### Agents definition: (1) Onboarding Personal Information Agent

The main goal of the Onboarding Personal Information Agent is to get the name and location of the customer. 

In [19]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding_Personal_Information_Agent",
    system_message='''You are a helpful customer onboarding agent,
    you work for a phone provider called ACME.
    Your job is to gather the customer's name and location.
    Do not ask for any other information, only ask about the customer's name and location.
    After the customer gives you their name and location, repeat them 
    and thank the user, and ask the user to answer with TERMINATE to move on to describing their issue.
    ''',
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower()
)

[autogen.oai.client: 10-01 10:52:55] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


### Agents definition: (2) Onboarding Issue Agent

The main goal of the Onboarding Issue Agent is to determine the issue the customer is facing with ACME's products.

In [11]:
onboarding_issue_agent = ConversableAgent(
    name="Onboarding_Issue_Agent",
    system_message='''You are a helpful customer onboarding agent,
    you work for a phone provider called ACME,
    you are here to help new customers get started with our product.
    Your job is to gather the product the customer use and the issue they currently 
    have with the product,
    Do not ask for other information.
    After the customer describes their issue, repeat it and add
    "Please answer with 'TERMINATE' if I have correctly understood your issue." ''',
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower()
)

[autogen.oai.client: 09-30 09:52:15] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


### Agents definition: (3) Customer Engagement Agent

The main goal of the **Customer Engagement Agent** is to interact with the customer based on the previously gathered information until a human agent is available to solve the customer's issue.

In [20]:
customer_engagement_agent = ConversableAgent(
    name="Customer_Engagement_Agent",
    system_message='''You are a helpful customer service agent.
    Your job is to gather customer's preferences on news topics.
    You are here to provide fun and useful information to the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

[autogen.oai.client: 10-01 10:58:03] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


### Customer Proxy Agent

The Customer Proxy Agent is a Conversable Agent that allows the human to play the role of the agent. It's not an LLM therefore `llm_config=False` and it has `human_input_mode="ALWAYS"` which means that you, the human, will be the customer through this agent.

In [12]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
)

### Next Step: Chat orchestration

Orchastrating how the chat will happen means that we will define in which order agents will interact and who'll interact with who when. To define this, we will use a list, that will contain several elements, each one corresponding to a chat. The chats will then happen in that specific order.

In [15]:
# defining a `chat` object
chats = [] # This is going to be our list of chats